In [10]:
from autoprotocol import Protocol

p = Protocol()

plate1 = p.ref( 'plate1', cont_type='96-deep', discard=True ) 
plate2 = p.ref( 'plate2', cont_type='96-deep', discard=True ) 

plate1.well(0).set_volume( '100:microliter' )

p.transfer( plate1.well(0), plate2.well(0), '10:microliter' )

import json
print json.dumps( p.as_dict(), indent=2 )

{
  "refs": {
    "plate2": {
      "new": "96-deep", 
      "discard": true
    }, 
    "plate1": {
      "new": "96-deep", 
      "discard": true
    }
  }, 
  "instructions": [
    {
      "groups": [
        {
          "transfer": [
            {
              "volume": "10.0:microliter", 
              "to": "plate2/0", 
              "from": "plate1/0"
            }
          ]
        }
      ], 
      "op": "pipette"
    }
  ]
}


In [30]:
# handle refs 
d = p.as_dict()
for index, plate in enumerate( d['refs'] ):
    print 'place plate %s with %s at index %d with xy coords %f, %f' % ( plate, d['refs'][ plate ], index, 69*index, 69*index ) 

place plate plate2 with {'new': '96-deep', 'discard': True} at index 0 with xy coords 0.000000, 0.000000
place plate plate1 with {'new': '96-deep', 'discard': True} at index 1 with xy coords 69.000000, 69.000000


In [65]:
# compile autoprotocol to gcode 
gcode = []
compiler_stack = []
for instruction in p.as_dict()['instructions']:
    #print instruction
    for group in instruction['groups']:
        #print group
        for transfer in group['transfer']:
            compiler_stack.append( transfer )

In [68]:
def a1_of_plate( plate ):
    return (0,0,0)

def offset_of_well( well ):
    return (0,0,0)

for transfer in compiler_stack:
    asp_vol = dis_vol = float( transfer['volume'].split( ':' )[0] )
    dis_xyz = tuple([ x + y for (x, y) in zip( a1_of_plate( transfer['to'] ), offset_of_well( transfer['to'].split( '/' )[-1] ) ) ])
    asp_xyz = tuple([ x + y for (x, y) in zip( a1_of_plate( transfer['from'] ), offset_of_well( transfer['from'].split( '/' )[-1] ) ) ] )
    
    gcode.append( 'G1 X%f Y%f Z%f F400' % dis_xyz )
    gcode.append( 'G1 A%f F500' % asp_vol )
    gcode.append( 'G1 X%f Y%f Z%f F400' % asp_xyz )
    gcode.append( 'G1 A-%f F500' % dis_vol )
    

print gcode

['G1 X0.000000 Y0.000000 Z0.000000 F400', 'G1 A10.000000 F500', 'G1 X0.000000 Y0.000000 Z0.000000 F400', 'G1 A10.000000 F500', 'G1 X0.000000 Y0.000000 Z0.000000 F400', 'G1 A10.000000 F500', 'G1 X0.000000 Y0.000000 Z0.000000 F400', 'G1 A10.000000 F500', 'G1 X0.000000 Y0.000000 Z0.000000 F400', 'G1 A10.000000 F500', 'G1 X0.000000 Y0.000000 Z0.000000 F400', 'G1 A-10.000000 F500']


In [64]:
[ x + y for (x, y) in zip( (1, 2, 3), (1, 2, 3) ) ] 

[2, 4, 6]